# Tensorflow

### author qhduan@memect.co

In [1]:
import pickle
import numpy as np
import tensorflow as tf

In [2]:
from data import X_train, X_test, y_train, y_test
from data import fit_vectorizer, fit_onehot
from data import batch_flow, test_batch_flow

训练集样本量：12126，测试集样本量：3032


In [3]:
tf.set_random_seed(0)

In [4]:
embedding_size = 128
PAD = ' ' # 句子不到max_len长度时的占位符
max_len = max(len(x) for x in X_train)
print('单个训练样本最大长度：{}'.format(max_len))

单个训练样本最大长度：14


In [5]:
vectorizer = fit_vectorizer(X_train, embedding_size, max_len, PAD)
onehot = fit_onehot(y_train)

In [6]:
learning_rate = 0.001
n_epoch = 20
hidden_size = 512
batch_size = 256
time_steps = max_len
input_size = embedding_size
target_size = len(onehot.feature_indices_)
print('time_steps', time_steps)
print('input_size', input_size)
print('target_size', target_size)

time_steps 14
input_size 128
target_size 2


In [7]:
test_batch_flow(X_train, y_train, batch_size, vectorizer, onehot, max_len, PAD)

(256, 14, 128) (256, 2)


In [8]:
X = tf.placeholder(tf.float32, [batch_size, time_steps, input_size, 1], name='X')
y = tf.placeholder(tf.float32, [batch_size, target_size], name='X')

In [9]:
pitch_1 = tf.Variable(tf.random_normal([5, 11, 1, 32]), name='pitch_1')
pitch_1_bias = tf.Variable(tf.random_normal([32]), name='pitch_1_bias')

pitch_2 = tf.Variable(tf.random_normal([3, 7, 32, 64]), name='pitch_2')
pitch_2_bias = tf.Variable(tf.random_normal([64]), name='pitch_2_bias')

In [10]:
conv_1 = tf.nn.relu(
    tf.nn.bias_add(
        tf.nn.conv2d(
            X, pitch_1, strides=[1, 1, 1, 1], padding='VALID'
        ),
        pitch_1_bias,
        name='bias_add_1'
    ),
    name='relu_1'
)

In [11]:
print(conv_1.get_shape())

(256, 10, 118, 32)


In [12]:
maxpool_1 = tf.nn.max_pool(
    conv_1,
    ksize=[1, 1, 8, 1],
    strides=[1, 1, 4, 1],
    padding='SAME',
    name='max_pool_1'
)

In [13]:
print(maxpool_1.get_shape())

(256, 10, 30, 32)


In [14]:
conv_2 = tf.nn.relu(
    tf.nn.bias_add(
        tf.nn.conv2d(
            maxpool_1, pitch_2, strides=[1, 1, 1, 1], padding='VALID'
        ),
        pitch_2_bias,
        name='bias_add_2'
    ),
    name='relu_2'
)

In [15]:
print(conv_2.get_shape())

(256, 8, 24, 64)


In [16]:
maxpool_2 = tf.nn.max_pool(
    conv_2,
    ksize=[1, 1, 4, 1],
    strides=[1, 1, 2, 1],
    padding='SAME',
    name='max_pool_2'
)

In [17]:
print(maxpool_2.get_shape())

(256, 8, 12, 64)


In [18]:
flatten = tf.reshape(maxpool_2, [batch_size, -1])

In [19]:
print(flatten.get_shape())

(256, 6144)


In [20]:
weight_1 = tf.Variable(tf.random_normal([int(flatten.get_shape()[1]), hidden_size]), name='weight_1')
bias_1 = tf.Variable(tf.random_normal([hidden_size]), name='bias_1')

weight_2 = tf.Variable(tf.random_normal([hidden_size, target_size]), name='weight_2')
bias_2 = tf.Variable(tf.random_normal([target_size]), name='bias_2')

In [21]:
full_connect_1 = tf.nn.relu(
    tf.add(
        tf.matmul(flatten, weight_1, name='matmul_1'),
        bias_1,
        name='add_1'
    ),
    name='relu_3'
)

In [22]:
full_connect_2 = tf.add(
    tf.matmul(full_connect_1, weight_2, name='matmul_2'),
    bias_2,
    name='add_2'
)

In [23]:
pred = full_connect_2

In [24]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        pred, y
    )
)

In [25]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [26]:
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [27]:
# 初始化所有变量
init = tf.global_variables_initializer()

In [28]:
# 本来是要关，不过CNN不用GPU真的好慢……
# disable GPU，关闭GPU支持
config = tf.ConfigProto(
#     device_count = {'GPU': 0}
)

In [29]:
with tf.Session(config=config) as sess:
    sess.run(init)
    for epoch in range(n_epoch + 1):
        costs = []
        accs = []
        for X_sample, y_sample in batch_flow(X_train, y_train, batch_size, vectorizer, onehot, max_len, PAD):
            feeds = {X: X_sample.reshape([batch_size, time_steps, input_size, 1]), y: y_sample}
            sess.run(train_step, feeds)
            c, acc = sess.run([cost, accuracy], feeds)
            costs.append(c)
            accs.append(acc)
        print('epoch {} cost: {:.4f} acc: {:.4f}'.format(
            epoch, np.mean(costs), np.mean(acc)
        ))
    # train
    costs = []
    accs = []
    for X_sample, y_sample in batch_flow(X_train, y_train, batch_size, vectorizer, onehot, max_len, PAD):
        feeds = {X: X_sample.reshape([batch_size, time_steps, input_size, 1]), y: y_sample}
        c, acc = sess.run([cost, accuracy], feeds)
        costs.append(c)
        accs.append(acc)
    print('train cost: {:.4f} acc: {:.4f}'.format(np.mean(costs), np.mean(acc)))
    # test
    costs = []
    accs = []
    for X_sample, y_sample in batch_flow(X_test, y_test, batch_size, vectorizer, onehot, max_len, PAD):
        feeds = {X: X_sample.reshape([batch_size, time_steps, input_size, 1]), y: y_sample}
        c, acc = sess.run([cost, accuracy], feeds)
        costs.append(c)
        accs.append(acc)
    print('test cost: {:.4f} acc: {:.4f}'.format(np.mean(costs), np.mean(acc)))

epoch 0 cost: 7313.8301 acc: 0.5742
epoch 1 cost: 4461.7412 acc: 0.5117
epoch 2 cost: 3760.0442 acc: 0.4805
epoch 3 cost: 3294.9746 acc: 0.5391
epoch 4 cost: 2994.0645 acc: 0.5508
epoch 5 cost: 2743.8994 acc: 0.5820
epoch 6 cost: 2545.1111 acc: 0.6016
epoch 7 cost: 2395.9573 acc: 0.6016
epoch 8 cost: 2322.5354 acc: 0.6523
epoch 9 cost: 2175.3826 acc: 0.6016
epoch 10 cost: 2154.6729 acc: 0.5117
epoch 11 cost: 2459.0969 acc: 0.6445
epoch 12 cost: 2042.2045 acc: 0.6562
epoch 13 cost: 1852.2140 acc: 0.6484
epoch 14 cost: 1938.1312 acc: 0.7109
epoch 15 cost: 1768.5895 acc: 0.7031
epoch 16 cost: 1531.2202 acc: 0.7070
epoch 17 cost: 1398.1149 acc: 0.6992
epoch 18 cost: 1335.4600 acc: 0.7148
epoch 19 cost: 1272.4141 acc: 0.7227
epoch 20 cost: 1255.8632 acc: 0.6953
train cost: 2050.3035 acc: 0.6953
test cost: 3083.1321 acc: 0.6172


要么过拟合，要么拟合不了，文本pitch的参数没什么经验～～不知道是不是程序写的有问题……应该不是吧